In [8]:
import pandas as pd
import os

## Prepare Utility Bill Inputs
This is heavily encouraged due to the error in the utility bills in this particular dataset

#### Option 1: Use Default Utility Rates
From ResStock Dataset 2024.2 Documentation

In [ ]:
####Prepare utility rates from ResStock 2024.2 documentation

##Inputs
#Electricity
fixed_elec_cost_monthly = 10
var_elec_cost_per_kwh = 0.136613018 #this is for PA
#look up other states here: https://nrel.github.io/ResStock.github.io/docs/resources/explanations/Issue_2024_2_Electricity_and_Energy_Bills.html

#Natural Gas
fixed_ng_cost_monthly = 11.25
var_ng_cost_per_kwh = 0.03268850625538582 #this is for PA
#look up other states here: https://github.com/NREL/resstock/blob/eussrr2/resources/data/simple_rates/State.tsv
#note NG is in $/therm at the link

#Fuel Oil
var_fo_cost_per_kwh = 0.07041247888248318 #this is for PA
#look up other states here: https://github.com/NREL/resstock/blob/eussrr2/resources/data/simple_rates/State.tsv
# ote propane and fuel oil are in $/gallon at the link

#Propane
var_propane_cost_per_kwh = 0.10145591100203372 #this is for PA
#look up other states here: https://github.com/NREL/resstock/blob/eussrr2/resources/data/simple_rates/State.tsv
#note propane and fuel oil are in $/gallon at the link)


#### Option 2: Use Custom Annual Utility Rates
This is currently specific to C2C DV and the units that project team provideed the utility rates in, but it can be readily customized for other rates and other units.

In [11]:
####Prepare the C2C Delaware Valley project utility rates

##Inputs
#Electricity
fixed_elec_cost_monthly = 10.56
var_elec_cost_per_kwh = 0.17404

#Natural Gas
fixed_ng_cost_monthly = 16.25
var_ng_cost_per_ccf = 1.495

#Fuel Oil
var_fo_cost_per_gal = 2.851

#Propane
var_propane_cost_per_gal = 2.723615385#3.199

##Unit Conversions
gal_fuel_oil_to_mbtu = 139/1000
gal_propane_to_mbtu = 91.6 / 1000
mbtu_to_kwh = 293.0710701722222
dol_per_ccf_to_dol_per_therm = 1/1.038 #$ per Ccf divided by 1.038 equals $ per therm https://www.eia.gov/tools/faqs/faq.php?id=45&t=8
therm_to_kwh = 29.307107017222222

var_ng_cost_per_kwh = var_ng_cost_per_ccf * (dol_per_ccf_to_dol_per_therm) * (1/therm_to_kwh)
var_fo_cost_per_kwh = var_fo_cost_per_gal * (1/gal_fuel_oil_to_mbtu) * (1/mbtu_to_kwh)
var_propane_cost_per_kwh = var_propane_cost_per_gal * (1/gal_propane_to_mbtu) * (1/mbtu_to_kwh)

print(var_ng_cost_per_kwh) #cf 0.034/kwh
print(var_fo_cost_per_kwh) #cf 0.070/kwh
print(var_propane_cost_per_kwh) #cf 0.101/kWh

0.0491440437526616
0.06998572515142275
0.10145591100203372


#### Assemble Utility Rates Inputs for Data Processing

In [12]:
rates_data_inputs = [
    ["out.bills_local.electricity.total.usd", fixed_elec_cost_monthly, var_elec_cost_per_kwh, ["out.electricity.total.energy_consumption.kwh"], 
     "out.x", "pivot", "Utility Bills", "Electricity", "Electricity Total", "Total"],
    ["out.bills_local.natural_gas.total.usd", fixed_ng_cost_monthly, var_ng_cost_per_kwh, ["out.natural_gas.total.energy_consumption.kwh"], 
     "out.x", "pivot", "Utility Bills", "Natural Gas", "Natural Gas Total", "Total"],
    ["out.bills_local.fuel_oil.total.usd", 0, var_fo_cost_per_kwh, ["out.fuel_oil.total.energy_consumption.kwh"], 
     "out.x", "pivot", "Utility Bills", "Fuel Oil", "Fuel Oil Total", "Total"],
    ["out.bills_local.propane.total.usd", 0, var_propane_cost_per_kwh, ["out.propane.total.energy_consumption.kwh"], 
     "out.x", "pivot", "Utility Bills", "Propane", "Total", "Total"],
    ["out.bills_local.all_fuels.total.usd", 0, 1, ["out.bills_local.electricity.total.usd", "out.bills_local.natural_gas.total.usd", "out.bills_local.fuel_oil.total.usd", "out.bills_local.propane.total.usd"], 
     "out.x", "pivot", "Utility Bill Totals", "Energy", "Total", "Total"],
    ["out.bills_local.electricity.total.usd.savings", 0, var_elec_cost_per_kwh, ["out.electricity.total.energy_consumption.kwh.savings"], 
     "out.x", "pivot", "Utility Bills", "Electricity", "Electricity Total", "Total"],
    ["out.bills_local.natural_gas.total.usd.savings", fixed_ng_cost_monthly, var_ng_cost_per_kwh, ["out.natural_gas.total.energy_consumption.kwh.savings"], 
     "out.x", "pivot", "Utility Bill Savings", "Natural Gas", "Natural Gas Total", "Total"],
    ["out.bills_local.fuel_oil.total.usd.savings", 0, var_fo_cost_per_kwh, ["out.fuel_oil.total.energy_consumption.kwh.savings"], 
     "out.x", "pivot", "Utility Bill Savings", "Fuel Oil", "Fuel Oil Total", "Total"],
    ["out.bills_local.propane.total.usd.savings", 0, var_propane_cost_per_kwh, ["out.propane.total.energy_consumption.kwh.savings"], 
     "out.x", "pivot", "Utility Bill Savings", "Propane", "Propane Total", "Total"],
    ["out.bills_local.all_fuels.total.usd.savings", 0, 1, ["out.bills_local.electricity.total.usd.savings", "out.bills_local.natural_gas.total.usd.savings", "out.bills_local.fuel_oil.total.usd.savings", "out.bills_local.propane.total.usd.savings"], 
     "out.x", "pivot", "Utility Bills Savings Totals", "Energy", "Total", "Total"]
]

one_year_bill_savings_col = "out.bills_local.all_fuels.total.usd.savings" #this is for NPV calculations, not utility bill calculations
rate_inputs_df = pd.DataFrame(rates_data_inputs, columns = ['column', 'fixed monthly cost', 'variable cost per kwh', 'col list for scaling', 'col_type', 'plan', 'Result Type', 'Fuel', 'End Use', 'End Use Category'])


## Prepare Upgrade Cost Inputs

In [14]:
##load ICF cost data and location info

cost_data_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Cost Data"

cost_file_location_value_match = ['Bucks County', 'Montgomery County', 'Chester County', 'Delaware County']
cost_file_location_field_match = ['in.county_name', 'in.county_name', 'in.county_name', 'in.county_name']
cost_file_name_list = ['NREL_Package Upgrade Costing_Doylestown_20250220.xlsx', 
                       'NREL_Package Upgrade Costing_Doylestown_20250220.xlsx',
                       'NREL_Package Upgrade Costing_West Chester_20250220.xlsx',
                       'NREL_Package Upgrade Costing_West Chester_20250220.xlsx']
cost_file_location_list = ['Doylestown', 'Doylestown', 'West Chester', 'West Chester']

cost_inputs = []
for file, location, location_field, location_value in zip(
    cost_file_name_list, cost_file_location_list, cost_file_location_field_match, cost_file_location_value_match):
    filepath = os.path.join(cost_data_folder, file)
    cost_data = pd.read_excel(filepath, sheet_name = "Python 1-16")
    cost_data["Location"] = location
    cost_data["Location Field Match"] = location_field
    cost_data["Location Value Match"] = location_value
    cost_inputs = cost_inputs + [cost_data]

first_costs_inputs_df = pd.concat(cost_inputs, ignore_index=True)

#the rest is handled in the Class - lots of hard-coding. Take a look if you are doing a new location.

## Set Up Other Necessary Inputs

In [15]:
##set up other variables need to process the data
#OK to skip anything that will not be run
resstock_results_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Data/2024.2/AMY2018"
downselect_row_fields = ["in.county_name"]
values_to_keep = [["Montgomery County", "Bucks County", "Chester County", "Delaware County"]]
testing_size = 5 #if you are testing, do it on just a few rows
col_plan_folder = 'C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Upgrade (4.3)'
col_plan_name = '2024-2 Col Plan including C2C DV Upgrades.csv'
outname_header = 'PA'
additional_columns_to_remove = []
wide_cols_also_long = ["out.emissions.all_fuels.lrmer_mid_case_15.co2e_kg", "out.bills_local.all_fuels.total.usd"]
wide_cols_also_long_names = ["Emissions", "Utility Bills Total"],

## Import the Class

In [16]:
##import the class 
from process_ResStock_2024_2 import process_ResStock_2024_2

IndentationError: expected an indented block (process_ResStock_2024_2.py, line 218)

## Process the Data
#### This loops through the list of data, one dataset at a time

In [ ]:
##process the data
up_list = ["baseline"]#["baseline", "upgrade02", "upgrade04", "upgrade07", "upgrade09", "upgrade12", "upgrade13", "upgrade16"]
processed_data = []
for up in up_list:
    print (up)
    resstock_file_name = "PA_" + up + "_metadata_and_annual_results.csv"
    outname = outname_header + "_" + up + "processed_results.csv"
    dataset_process = process_ResStock_2024_2(resstock_results_folder, resstock_file_name)
    dataset_process.downselect_rows(downselect_row_fields, values_to_keep)
    #dataset_process.make_data_testing_size(testing_size) #shrink the data to a smaller number of rows for testing
    dataset_process.load_and_process_column_plan(col_plan_folder, col_plan_name)
    dataset_process.add_local_bills(rate_inputs_df)
    dataset_process.add_first_costs(first_costs_inputs_df)
    #dataset_process.add_additional_wide_fields(addl_wide_fields_df) #None needed for C2C DV
    dataset_process.downselect_columns(additional_columns_to_remove)
    dataset_process.pivot_data()
    #dataset_process.add_additional_long_fields(addl_long_fields_df) #None needed for C2C DV
    dataset_process.categorize_outputs()
    dataset_process.wide_fields_also_long(wide_cols_also_long, wide_cols_also_long_names)
    dataset_process.add_weighted_values_column()
    processed_data = processed_data + [dataset_process.data_long]


baseline
23059
4097


## Save Processed Data

In [ ]:
#save multiple sets of processed data 

##set destination
output_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Upgrade (4.3)"
output_file_name = "ResStock_long_v5_4counties.csv"
output_path = os.path.join(output_folder, output_file_name)

#save first set of data, with headers
processed_data[0].to_csv(output_path, header = True, index = False)
print(processed_data[0]["upgrade"][0])

#save remaining data, appending it to the same file. 
#This works since all the datasets are forced to have the same set of columns in the same order.
for dataset in processed_data[1:]:
    print(dataset["upgrade"][0])
    dataset.to_csv(output_path, header = False, index = False, mode = 'a')